# BeeAI Framework Basics

These examples show some of the basic usage patterns of BeeAI in Python.  They gradually build in complexity to give a rounded overview.

## Prompt Templates

One of the most basic constructs provided by the BeeAI framework is the `PromptTemplate`. Using a PromptTemplate you can incorporate data into a prompt before sending it a language model.
Prompt templates are based on the mustache templating language.

The following example shows you how to create a RAG (Retrieval Augmented Generation) template and apply the template to your data to generate a prompt.

In [ ]:
import warnings

from pydantic import BaseModel

from beeai_framework.utils.templates import PromptTemplate

warnings.simplefilter("ignore", UserWarning)


# The input schema model: Defines the structure of the input data that can passed to the template
class RAGTemplateInput(BaseModel):
    question: str
    context: str


# Define the template
rag_template: PromptTemplate = PromptTemplate(
    schema=RAGTemplateInput,
    template="""
Context: {{context}}
Question: {{question}}

Provide a concise answer based on the context. Avoid statements such as 'Based on the context' or 'According to the context' etc. """,
)

# Render the template using an instance of the input model
prompt = rag_template.render(
    RAGTemplateInput(
        question="What is the capital of France?",
        context="France is a country in Europe. Its capital city is Paris, known for its culture and history.",
    )
)

print(prompt)

## More complex templates

That was a simple template but the `PromptTemplate` class can also be used to render more complex objects and include conditional logic.

The next example is a template that includes a question and a set of complex search results.

In [ ]:
from pydantic import BaseModel

from beeai_framework.utils.templates import PromptTemplate


class SearchResult(BaseModel):
    title: str
    url: str
    content: str


class SearchTemplateInput(BaseModel):
    question: str
    results: list[SearchResult]


# Define the template
search_template: PromptTemplate = PromptTemplate(
    schema=SearchTemplateInput,
    template="""
Search results:
{{#results.0}}
{{#results}}
Title: {{title}}
Url: {{url}}
Content: {{content}}
{{/results}}
{{/results.0}}

Question: {{question}}
Provide a concise answer based on the search results provided.""",
)

# Render the template using an instance of the input model
prompt = search_template.render(
    SearchTemplateInput(
        question="What is the capital of France?",
        results=[
            SearchResult(
                title="France",
                url="https://en.wikipedia.org/wiki/France",
                content="France is a country in Europe. Its capital city is Paris, known for its culture and history.",
            )
        ],
    )
)

print(prompt)

## The ChatModel

Once you have your PromptTemplate you can start to think about prompting a model. BeeAI supports a variety of LLM's that you can use via the `ChatModel` interface. 

In this section we will use the `IBM Granite 3.1 8B` language model via the Ollama provider.

[How to run Granite 3.1 using Ollama](https://www.ibm.com/granite/docs/run/granite-on-mac/granite/).

Before creating a ChatModel we need to briefly cover Messages. The `ChatModel` interface operates using messages. Using messages you can represent a chat between the user and the assistant (the LLM) which is a convenient interaction method. Lets start by creating a `UserMessage` to say hello and ask a simple question.

In [ ]:
from beeai_framework.backend.message import UserMessage

# Create a user message to start a chat with the model
user_message = UserMessage(content="Hello! Can you tell me what is the capital of France?")

We can now create a `ChatModel` and send this message to Granite.

In [ ]:
from beeai_framework.backend.chat import ChatModel, ChatModelInput, ChatModelOutput

# Create a ChatModel to interface with granite3.1-dense:8b on a local ollama
model = ChatModel.from_name("ollama:granite3.1-dense:8b")

output: ChatModelOutput = await model.create(ChatModelInput(messages=[user_message]))

print(output.get_text_content())

## Memory 
The model has provided a response! We can now start to build up a `Memory`. Memory is just a convenient way of storing a set of messages that can be considered as the history of the dialog between the user and the llm.

In this next example we will construct a memory from our existing messages and add a new user message. Notice that the new message can implicitly refer to content from prior messages. Internally the `ChatModel` formats all the messages and sends them to the LLM.

In [ ]:
from beeai_framework.backend.message import AssistantMessage
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory

memory = UnconstrainedMemory()

await memory.add_many(
    [
        user_message,
        AssistantMessage(content=output.get_text_content()),
        UserMessage(content="If you had to recommend one thing to do there, what would it be?"),
    ]
)

output: ChatModelOutput = await model.create(ChatModelInput(messages=memory.messages))

print(output.get_text_content())

## Combining Templates and Messages

If you would like to use a `PromptTemplate` from earlier with the Granite ChatModel, you can render the template and then put the content into a Message.

In [ ]:
# Some context that the model will use to provide an answer. Source wikipedia: https://en.wikipedia.org/wiki/Ireland
context = """The geography of Ireland comprises relatively low-lying mountains surrounding a central plain, with several navigable rivers extending inland.
Its lush vegetation is a product of its mild but changeable climate which is free of extremes in temperature.
Much of Ireland was woodland until the end of the Middle Ages. Today, woodland makes up about 10% of the island,
compared with a European average of over 33%, with most of it being non-native conifer plantations.
The Irish climate is influenced by the Atlantic Ocean and thus very moderate, and winters are milder than expected for such a northerly area,
although summers are cooler than those in continental Europe. Rainfall and cloud cover are abundant.
"""

# Lets reuse our RAG template from earlier!
prompt = rag_template.render(RAGTemplateInput(question="How much of Ireland is forested?", context=context))

output: ChatModelOutput = await model.create(ChatModelInput(messages=[UserMessage(content=prompt)]))

print(output.get_text_content())

## Structured Outputs

Sometimes (often!) you will want llm output in a specific format. This will allow you to interface the llm with your code in a reliable manner i.e. if you want the llm to produce the input to a function or tool. To achieve this you can use structured output.

In the example below I want Granite to generate a character using a very specific format.

In [ ]:
from typing import Literal

from pydantic import Field


class CharacterSchema(BaseModel):
    name: str = Field(description="The name of the character.")
    occupation: str = Field(description="The occupation of the character.")
    species: Literal["Human", "Insectoid", "Void-Serpent", "Synth", "Ethereal", "Liquid-Metal"] = Field(
        description="The race of the character."
    )
    back_story: str = Field(description="Brief backstory of this character.")


user_message = UserMessage(
    "Crete a fantasy sci-fi character for my new game. This character will be the main protagonist."
)
response = await model.create_structure(
    {
        "schema": CharacterSchema,
        "messages": [user_message],
    }
)

print(response.object)

## System Prompts

The system prompt or SystemMessage is a special message type that can influence the general behavior of an LLM. If you would like to influence an LLM in a general manner you can include a SystemMessage.

In the example below we add a system message that influences the LLM to speak like a pirate!

In [ ]:
from beeai_framework.backend.message import SystemMessage

system_message = SystemMessage(content="You are pirate. You always respond using pirate slang.")
user_message = UserMessage(content="What is a baby hedgehog called?")
output: ChatModelOutput = await model.create(ChatModelInput(messages=[system_message, user_message]))

print(output.get_text_content())

## Building an Agent

You are now ready to build you first agent. Move on to [workflows.ipynb](workflows.ipynb).
